In [1]:
def odom_callback(msg):
    
    odom.pose.pose.position.x = msg.pose.pose.position.x
    odom.pose.pose.position.y = msg.pose.pose.position.y

In [2]:
def get_var():
	client.call(var)
	
	# Information about the status of the goal
	var.response.s = var.request.succ
	var.response.re = var.request.rej
	
	# Information about the minimum distances on the right, in front and on the left
	var.response.ri = var.request.right
	var.response.c = var.request.center
	var.response.l = var.request.left
	
	# Information about the keyboard command
	var.response.d = var.request.dir
	
	# Information about which controlling modality to use
	var.response.m1 = var.request.mod1
	var.response.m2 = var.request.mod2

In [3]:
def auto_move():
	canc = GoalID()
	canc = {}
	
	print("You chose modality 1! \n")
	get_var()
	
	# Ask the user a new target position
	print("Set x and y of new target position: ")
	box = widgets.HBox(
		[
			widgets.FloatText(
				value = None,
				description = 'x: ',
				disabled = False
			),
			widgets.FloatText(
				value = None,
				description = 'y: ',
				disabled = False
			)
		]
	)
    
	def publish_target(change):
		target.goal.target_pose.pose.position.x = box[0].value
		target.goal.target_pose.pose.position.y = box[1].value
		pub_goal.publish(target)
        
	display(box, output)
	box.observe(publish_target, name='value')
    
	# Drive the robot towards target
	while (var.response.s!=1):
		var.request.mod1 = 0
		client.call(var)
	
	# Control if the target is unreachable and eventually cancel it, otherwise ask the user 
	#  if he/she wants to cancel it
	if (var.response.re==1): # target is unreachable
		print("Goal cannot be reached!")
		pub_canc.publish(canc)
	else: # asking the user if target has to be cancelled
		choose_to_canc = widgets.RadioButtons(
			options = ['Yes', 'No'],
			layout = {'width': 'max-content'},
			description = 'Do you want to cancel this goal?',
			disabled = False,
			value = None
		)
        
		def cancelling(change):
			if (choose_to_canc.value=='Yes'): # the user wants to cancel it
				pub_canc.publish(canc)
				print("Goal cancelled")
			else:
				print("Ok, going towards the goal")
                
		display(choose_to_canc, output)
		choose_to_canc.observe(cancelling, name=['value'])

In [4]:
def drive_alone():
    
	print("You chose modality 2! \n")
	
	get_var()
	
	print("Use keyboard to move the robot:")
	while (var.response.d!='q'):
		var.request.mod2 = 1
		client.call(var)

In [5]:
def drive_assistance():
    
	print("You chose modality 3! \n")
	
	client(var)
	
	print("Use keyboard to move the robot:")
	while (var.response.d!='q'):
		var.request.mod1 = 0
		client.call(var)
		var.request.mod2 = 1
		client.call(var)

In [6]:
def ui_decide():
	
	choose_mod = widgets.RadioButtons(
		options = ['robot moves autonomously to goal', 'drive the robot to goal', 'drive the robot with assistance to goal'],
		layout = {'width': 'max-content'},
		description = 'Choose:',
		disabled = False,
        value = None
	)
    
	def choosing(change):
		if (choose_mod.value=='robot moves autonomously to goal'):
			auto_move()
		elif (choose_mod.value=='drive the robot to goal'):
 			drive_alone()
		elif (choose_mod.value=='drive the robot with assistance to goal'):
			drive_assistance()
		else:
			print("This modality doesn't exist.")
        
	display(choose_mod, output)
	choose_mod.observe(choosing, names=['value'])

In [7]:
def plot_robot_data():

    x_odom = odom.pose.pose.position.x
    y_odom = odom.pose.pose.position.y
    ax_odom.plot(x_odom,y_odom)
    plt.show()

In [8]:
def plot_target_data():
    
    #%matplotlib widget
    x_target = target.goal.target_pose.pose.position.x
    y_target = target.goal.target_pose.pose.position.y
    fig_target = plt.figure()
    ax_target = fig_target.add_axes([0,0,1,1])
    ax_target.plot(x_target,y_target,'*')
    ax_target.set_title("Target position")
    ax_target.set_xlabel("x [m]")
    ax_target.set_ylabel("y [m]")

In [9]:
def interface():
    rospy.init_node("user_interface")
    
    global pub_goal
    global pub_canc
    global client
    global sub_odom
    
    pub_goal = rospy.Publisher("/move_base/goal", MoveBaseActionGoal, queue_size=10)
    pub_canc = rospy.Publisher("/move_base/cancel", GoalID, queue_size=10)
    client = rospy.ServiceProxy("/service", Callbacks_srv)
    sub_odom = rospy.Subscriber("/odom", Odometry, odom_callback)    
    
    ui_decide()
    #plot_robot_data()
    #plot_target_data()

In [10]:
import sys
sys.path.append('/root/ros_ws/src/RT1-III_assignment')

import final_assignment
import rospy
import matplotlib.pyplot as plt
import ipywidgets as widgets

from nav_msgs.msg import Odometry
from move_base_msgs.msg import MoveBaseActionGoal
from actionlib_msgs.msg import GoalID
from final_srv.srv import Callbacks_srv

var = Callbacks_srv()
odom = Odometry()
target = MoveBaseActionGoal()

output = widgets.Output()

# Modo 2:
# fig_odom = plt.figure()
# ax_odom = fig_odom.add_axes([0,0,5,5])
# ax_odom.set_title("Robot position")
# ax_odom.set_xlabel("x [m]")
# ax_odom.set_ylabel("y [m]")

# %matplotlib widget
# fig_odom = plt.figure()
# ax_odom = fig_odom.add_axes([0,0,2,2])
# ax_odom.set_title("Robot position")
# ax_odom.set_xlabel("x [m]")
# ax_odom.set_ylabel("y [m]")

interface()

RadioButtons(description='Choose:', layout=Layout(width='max-content'), options=('robot moves autonomously to …

Output()

You chose modality 3! 



TypeError: Invalid number of arguments, args should be ['succ', 'rej', 'right', 'center', 'left', 'dir', 'mod1', 'mod2'] args are(<final_srv.srv._Callbacks_srv.Callbacks_srv object at 0x7f1e5711e070>,)